In [1]:
# !pip install git+git://github.com/jsignell/dask-geopandas.git

  Cloning git://github.com/jsignell/dask-geopandas.git to /tmp/pip-req-build-wmkzdt98
  Created wheel for dask-geopandas: filename=dask_geopandas-0+untagged.7.gcb73333-py3-none-any.whl size=13073 sha256=771843a4443b04c9cb4ebf5467f7d81c367acbcc1a432b4d69d2341add53952f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdqpagb6/wheels/ff/55/cb/2b4f9123cf14297d989da457790c5d863cda30fa41ab22d763
Successfully built dask-geopandas


In [1]:
from dask.distributed import Client, LocalCluster
import dask_geopandas as dg
import geopandas
import pygeos
import numpy as np
import pandas as pd
import collections

/opt/conda/lib/python3.7/site-packages/pyarrow/compat.py:25: FutureWarning: pyarrow.compat has been deprecated and will be removed in a future release
  "future release", FutureWarning)


In [2]:
client = Client(LocalCluster(n_workers=14))
client

Client Scheduler: tcp://127.0.0.1:38697 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 14 Cores: 28 Memory: 84.28 GB


In [3]:
df = geopandas.read_parquet("../../urbangrammar_samba/spatial_signatures/tessellation/tess_0.pq")

In [4]:
ddf = dg.from_geopandas(df, npartitions=14)

In [5]:
def get_bbox(series):
    return series.apply(lambda g: g.minimum_rotated_rectangle)

bbox = ddf.geometry.map_partitions(get_bbox, meta=geopandas.GeoSeries())

In [6]:
from momepy_utils import solar_orientation_poly

def _solar_orientation_poly(series):
    return series.apply(lambda g: solar_orientation_poly(g))

ddf['ssbOri'] = bbox.map_partitions(lambda s: _solar_orientation_poly(s), meta='float')

In [7]:
%time ddf.compute()

CPU times: user 1.46 s, sys: 168 ms, total: 1.62 s
Wall time: 7.69 s


,uID,geometry,ssbOri
2326419,2326419,"POLYGON ((359111.400 434416.180, 359123.470 43...",9.593134
2326423,2326423,"POLYGON ((359130.950 434416.050, 359122.160 43...",8.667776
2326424,2326424,"POLYGON ((359130.610 434419.410, 359145.280 43...",8.904470
2326425,2326425,"POLYGON ((359160.490 434474.880, 359159.490 43...",4.136699
2326426,2326426,"POLYGON ((359156.180 434436.290, 359167.620 43...",12.661314
...,...,...,...
2573206,2573206,"POLYGON ((335422.740 427198.940, 335425.690 42...",22.918823
2573207,2573207,"POLYGON ((335421.370 427071.200, 335439.210 42...",23.453925
2573208,2573208,"POLYGON ((335432.380 427227.080, 335438.650 42...",22.915106
2573209,2573209,"POLYGON ((335464.150 427074.040, 335467.790 42...",22.988717


In [8]:
client.close()

## vectorized Queen contiguity

In [20]:
df = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
df = df.explode()

In [6]:
import itertools as it
import collections


def _get_verts(shape):
    return _get_boundary_points(shape)

def _get_boundary_points(shape):
    """
    Recursively handle polygons vs. multipolygons to
    extract the boundary point set from each. 
    """
    if shape.type.lower() == 'polygon':
        shape = shape.boundary
        return _get_boundary_points(shape)
    elif shape.type.lower() == 'linestring':
        return list(map(tuple, list(zip(*shape.coords.xy))))
    elif shape.type.lower() == 'multilinestring':
        return list(it.chain(*(list(zip(*shape.coords.xy))
                                 for shape in shape)))
    elif shape.type.lower() == 'multipolygon':
        return list(it.chain(*(_get_boundary_points(part.boundary) 
                               for part in shape)))
    else:
        raise TypeError('Input shape must be a Polygon, Multipolygon, LineString, '
                        ' or MultiLinestring and was '
                        ' instead: {}'.format(shape.type))

In [72]:
collection = df.geometry.values

In [111]:
numPoly = len(collection)

w = {}
for i in range(numPoly):
    w[i] = set()

geoms = []  # list of coordinate tuples
offsets = []  # identificators of origin geoms for tuples
c = 0  # PolyID Counter


for n in range(numPoly):
    verts = _get_verts(collection[n])
    offsets += [c] * len(verts)
    geoms += (verts)
    c += 1

items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

shared_vertices = []
for item, location in list(items.items()):
    if len(location) > 1:
        shared_vertices.append(location)

for vert_set in shared_vertices:
    for v in vert_set:
        w[v] = w[v] | vert_set
        try:
            w[v].remove(v)
        except:
            pass

In [83]:
%%timeit
numPoly = len(collection)

w = {}
for i in range(numPoly):
    w[i] = set()

geoms = []  # list of coordinate tuples
offsets = []  # identificators of origin geoms for tuples
c = 0  # PolyID Counter


for n in range(numPoly):
    verts = _get_verts(collection[n])
    offsets += [c] * len(verts)
    geoms += (verts)
    c += 1

85.2 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
w = {}
for i in range(numPoly):
    w[i] = set()

In [80]:
geoms = []
offsets = []
c = 0  # PolyID Counter

In [81]:
%%timeit
for n in range(numPoly):
    verts = _get_verts(collection[n])
    offsets += [c] * len(verts)
    geoms += (verts)
    c += 1

UnboundLocalError: local variable 'offsets' referenced before assignment

In [35]:
offsets

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,


In [36]:
geoms

[(180.0, -16.067132663642447),
 (180.0, -16.555216566639196),
 (179.36414266196414, -16.801354076946883),
 (178.72505936299711, -17.01204167436804),
 (178.59683859511713, -16.639150000000004),
 (179.0966093629971, -16.433984277547403),
 (179.4135093629971, -16.379054277547404),
 (180.0, -16.067132663642447),
 (178.12557, -17.50481),
 (178.3736, -17.33992),
 (178.71806, -17.62846),
 (178.55271, -18.15059),
 (177.93266000000003, -18.28799),
 (177.38146, -18.16432),
 (177.28504, -17.72465),
 (177.67087, -17.381140000000002),
 (178.12557, -17.50481),
 (-179.79332010904864, -16.020882256741224),
 (-179.9173693847653, -16.501783135649397),
 (-180.0, -16.555216566639196),
 (-180.0, -16.067132663642447),
 (-179.79332010904864, -16.020882256741224),
 (33.90371119710453, -0.9500000000000001),
 (34.07261999999997, -1.0598199999999451),
 (37.69868999999994, -3.0969899999999484),
 (37.7669, -3.6771200000000004),
 (39.20222, -4.67677),
 (38.74053999999995, -5.9089499999999475),
 (38.79977000000008, 

In [39]:
items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

In [40]:
items

defaultdict(set,
            {(180.0, -16.067132663642447): {0, 100, 200},
             (180.0, -16.555216566639196): {0, 100, 200},
             (179.36414266196414, -16.801354076946883): {0, 100, 200},
             (178.72505936299711, -17.01204167436804): {0, 100, 200},
             (178.59683859511713, -16.639150000000004): {0, 100, 200},
             (179.0966093629971, -16.433984277547403): {0, 100, 200},
             (179.4135093629971, -16.379054277547404): {0, 100, 200},
             (178.12557, -17.50481): {1, 101, 201},
             (178.3736, -17.33992): {1, 101, 201},
             (178.71806, -17.62846): {1, 101, 201},
             (178.55271, -18.15059): {1, 101, 201},
             (177.93266000000003, -18.28799): {1, 101, 201},
             (177.38146, -18.16432): {1, 101, 201},
             (177.28504, -17.72465): {1, 101, 201},
             (177.67087, -17.381140000000002): {1, 101, 201},
             (-179.79332010904864, -16.020882256741224): {2, 102, 202},
         

In [ ]:
# new implementation

In [92]:
%%timeit
geoms = pygeos.get_coordinates(collection.data)
indices = pygeos.get_num_coordinates(collection.data)
offsets = []
for i, c in enumerate(indices):
    offsets += [i] * c
geoms = np.array((map(tuple, geoms)))

19.7 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
geoms.shape

(10654, 2)

In [66]:
unique, counts = np.unique(geoms, axis=0, return_counts=True)

In [67]:
shared = unique[counts > 1]

In [69]:
len(shared)

2777

In [85]:
geoms

[(180.0, -16.067132663642447),
 (180.0, -16.555216566639196),
 (179.36414266196414, -16.801354076946883),
 (178.72505936299711, -17.01204167436804),
 (178.59683859511713, -16.639150000000004),
 (179.0966093629971, -16.433984277547403),
 (179.4135093629971, -16.379054277547404),
 (180.0, -16.067132663642447),
 (178.12557, -17.50481),
 (178.3736, -17.33992),
 (178.71806, -17.62846),
 (178.55271, -18.15059),
 (177.93266000000003, -18.28799),
 (177.38146, -18.16432),
 (177.28504, -17.72465),
 (177.67087, -17.381140000000002),
 (178.12557, -17.50481),
 (-179.79332010904864, -16.020882256741224),
 (-179.9173693847653, -16.501783135649397),
 (-180.0, -16.555216566639196),
 (-180.0, -16.067132663642447),
 (-179.79332010904864, -16.020882256741224),
 (33.90371119710453, -0.9500000000000001),
 (34.07261999999997, -1.0598199999999451),
 (37.69868999999994, -3.0969899999999484),
 (37.7669, -3.6771200000000004),
 (39.20222, -4.67677),
 (38.74053999999995, -5.9089499999999475),
 (38.79977000000008, 

In [87]:
geoms

array([[180.        , -16.06713266],
       [180.        , -16.55521657],
       [179.36414266, -16.80135408],
       ...,
       [ 31.88145   ,   3.55827   ],
       [ 31.24556   ,   3.7819    ],
       [ 30.83385242,   3.5091716 ]])

In [110]:
np.where(np.all(geoms == shared[2], axis=1))[0]

array([3034, 3057])

In [112]:
shared_vertices

[{3, 70, 279},
 {3, 70},
 {3, 70},
 {3, 70},
 {3, 70},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 149},
 {3, 148, 149},
 {3, 148},
 {3, 148},
 {3, 148},
 {3, 147, 148},
 {3, 147},
 {3, 147},
 {3, 147},
 {3, 68, 147},
 {3, 68},
 {3, 68},
 {3, 68},
 {3, 68},
 {3, 68},
 {3, 68, 153},
 {3, 153},
 {3, 153},
 {3, 153},
 {3, 153},
 {3, 153},
 {3, 153},
 {3, 153, 280},
 {3, 280},
 {3, 280},
 {3, 280},
 {3, 279, 280},
 {3, 279},
 {3, 279},
 {4, 160, 273},
 {4, 160},
 {4, 130, 160},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 130},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {4, 273},
 {5, 35},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 43},
 {5, 35}

In [ ]:
numPoly = len(collection)

w = {}
for i in range(numPoly):
    w[i] = set()

geoms = pygeos.get_coordinates(collection.data)
indices = pygeos.get_num_coordinates(collection.data)

offsets = []
for i, c in enumerate(indices):
    offsets += [i] * c
geoms = np.array((map(tuple, geoms)))

items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

shared_vertices = []
for item, location in list(items.items()):
    if len(location) > 1:
        shared_vertices.append(location)

for vert_set in shared_vertices:
    for v in vert_set:
        w[v] = w[v] | vert_set
        try:
            w[v].remove(v)
        except:
            pass

In [123]:
np.array([[1.1,1.1][1.1,1.1]])

TypeError: list indices must be integers or slices, not tuple

In [121]:
ddf

,uID,geometry,ssbOri
npartitions=14,,,
2326419,int64,geometry,float64
2370499,...,...,...
...,...,...,...
2565137,...,...,...
2573210,...,...,...


In [124]:
def get_coordinates(series):
    return pygeos.get_coordinates(series.values.data)

geoms = ddf.geometry.map_partitions(get_coordinates, meta=np.array([[1.1,1.1], [1.1,1.1]]))

def get_indices(series):
    return pygeos.get_coordinates(series.values.data)

indices = ddf.geometry.map_partitions(get_indices, meta=np.array([1, 1]))

In [125]:
geoms.compute()

KeyboardInterrupt: 

In [ ]:
numPoly = len(ddf)

w = {}
for i in range(numPoly):
    w[i] = set()

def get_coordinates(series):
    return pygeos.get_coordinates(series.values.data)

geoms = ddf.buildings.map_partitions(get_coordinates).compute()

def get_indices(series):
    return pygeos.get_num_coordinates(series.values.data)

indices = ddf.buildings.map_partitions(get_indices, meta=np.array([1, 1])).compute()

offsets = []
for i, c in enumerate(indices):
    offsets += [i] * c
geoms = np.array((map(tuple, geoms)))

items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

shared_vertices = []
for item, location in list(items.items()):
    if len(location) > 1:
        shared_vertices.append(location)

for vert_set in shared_vertices:
    for v in vert_set:
        w[v] = w[v] | vert_set
        try:
            w[v].remove(v)
        except:
            pass

In [145]:
df = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [127]:
ddf = dg.from_geopandas(df, npartitions=14)

In [142]:
def get_coordinates(series):
    return pygeos.get_coordinates(series.values.data), pygeos.get_num_coordinates(series.values.data)

geoms = ddf.geometry.map_partitions(get_coordinates)

In [143]:
g = geoms.compute()

In [150]:
len(ddf)

112951

In [5]:
%%time
numPoly = len(ddf)

w = {}
for i in range(numPoly):
    w[i] = set()

def get_coordinates(series):
    return pygeos.get_coordinates(series.values.data)

geoms = ddf.geometry.map_partitions(get_coordinates).compute()

def get_indices(series):
    return pygeos.get_num_coordinates(series.values.data)

indices = ddf.geometry.map_partitions(get_indices).compute()

offsets = []
for i, c in enumerate(indices):
    offsets += [i] * c
geoms = list(map(tuple, geoms))

items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

shared_vertices = []
for item, location in list(items.items()):
    if len(location) > 1:
        shared_vertices.append(location)

for vert_set in shared_vertices:
    for v in vert_set:
        w[v] = w[v] | vert_set
        try:
            w[v].remove(v)
        except:
            pass

CPU times: user 11.6 s, sys: 1.18 s, total: 12.8 s
Wall time: 13.6 s


In [7]:
%%time
collection = df.geometry.values
numPoly = len(df)

w = {}
for i in range(numPoly):
    w[i] = set()

geoms = []  # list of coordinate tuples
offsets = []  # identificators of origin geoms for tuples
c = 0  # PolyID Counter


for n in range(numPoly):
    verts = _get_verts(collection[n])
    offsets += [c] * len(verts)
    geoms += (verts)
    c += 1

CPU times: user 44.4 s, sys: 6.81 s, total: 51.2 s
Wall time: 44.8 s


In [8]:
%%time
items = collections.defaultdict(set)
for i, vertex in enumerate(geoms):
    items[vertex].add(offsets[i])

shared_vertices = []
for item, location in list(items.items()):
    if len(location) > 1:
        shared_vertices.append(location)

for vert_set in shared_vertices:
    for v in vert_set:
        w[v] = w[v] | vert_set
        try:
            w[v].remove(v)
        except:
            pass

CPU times: user 18.6 s, sys: 668 ms, total: 19.2 s
Wall time: 18.9 s


In [ ]:
client.close()